In [116]:
from selenium import webdriver
import time
import requests 
import io
from PIL import Image
import hashlib
import os
from selenium.webdriver.support.ui import WebDriverWait

DRIVER_PATH = r'C:\Users\Lenovo\Desktop\PhotosScrapper\chromedriver.exe'

In [127]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    delay = 10 # seconds
    # build the google query
    search_url = "https://www.bing.com/images/search?q={q}"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        #scroll_to_end(wd)

        # get all image thumbnail results
        wd.implicitly_wait(20)
        thumbnail_results = wd.find_elements_by_css_selector("img.mimg")
        #href= thumbnail_results.get_attribute('href')
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            #try:
            #    img.click()
            #    time.sleep(sleep_between_interactions)
            #except Exception:
            #    continue

            # extract image urls    
            #actual_images = wd.find_elements_by_css_selector('div.imgContainer nofocus')
            #for actual_image in actual_images:
            if img.get_attribute('src'):
                image_urls.add(img.get_attribute('src'))

            image_count = len(image_urls)
            print(image_count)
            print(max_links_to_fetch)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            t
            load_more_button = WebDriverWait(wd, delay).until(wd.find_element_by_css_selector(".ksb"))
            if load_more_button:
                wd.execute_script("document.querySelector('.ksb').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [118]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [129]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=2):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [120]:
import pandas as pd
games = pd.read_csv(r'C:\Users\Lenovo\Desktop\GIT\GamersHub\WebScraping\export_dataframe.csv',sep=';')

In [130]:
path = r'C:\Users\Lenovo\Desktop\GIT\GamersHub\WebScraping\PhotosScrapper\images'

folders = os.listdir(path)

for name in games['Titile']:
    name = name.replace(":", " ")
    name = name + " screenshot"
    folder_name = name.replace(" ", "_").lower()
    print(folder_name)
    if folder_name not in folders:
        search_term = name

        search_and_download(
            search_term=search_term,
            driver_path=DRIVER_PATH
        )

fallout_4_screenshot
divinity__original_sin_ii_-_definitive_edition_screenshot
outlast_2_screenshot
borderlands_3_screenshot
counter-strike__global_offensive_screenshot
league_of_legends_screenshot
citadel__forged_with_fire_screenshot
luigi's_mansion_3_screenshot
god_of_war_screenshot
the_witcher_2__assassins_of_kings_screenshot
minecraft_screenshot
batman__arkham_knight_screenshot
farming_simulator_17_screenshot
brawl_stars_screenshot
metro_exodus_screenshot
diablo_iii_screenshot
forza_horizon_4_screenshot
grand_theft_auto__san_andreas_screenshot
dying_light_screenshot
shadow_of_the_tomb_raider_screenshot
elex_screenshot
need_for_speed__most_wanted_(2005)_screenshot
days_gone_screenshot
hearts_of_iron_iv_screenshot
grand_theft_auto_iv_screenshot
fallout__new_vegas_screenshot
assassin's_creed_origins_screenshot
the_witcher_3__blood_and_wine_screenshot
grand_theft_auto__vice_city_screenshot
the_last_of_us_screenshot
s.t.a.l.k.e.r.__shadow_of_chernobyl_screenshot
soma_screenshot
far_cry_

SUCCESS - saved https://www.bing.com/th?id=OIP.2tesqzCV4SQ1wyIrdQL3qQHaEK&w=236&h=160&c=7&o=5&pid=1.7 - as ./images\ark__survival_evolved_screenshot\c0569c21e4.jpg
assassin's_creed_iii_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.s6p_5v23pEpUhN11ZpEDzgHaEK&w=295&h=165&c=7&o=5&pid=1.7 - as ./images\assassin's_creed_iii_screenshot\2b2a98c8d6.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.US6IZktkdy3i-G6b3Ji_gwHaEK&w=295&h=165&c=7&o=5&pid=1.7 - as ./images\assassin's_creed_iii_screenshot\5f76ce195c.jpg
grim_dawn_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.PQVHJf2Y3_tqSvjPf91zmQHaFj&w=253&h=181&c=7&o=5&pid=1.7 - as ./images\grim_dawn_screenshot\dbf5d2a89e.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.6Uqij5siYBCXNLaOOH3nvwHaEK&w=300&h=168&c=7&o=5&pid=1.7 - as ./images\grim_dawn_s

SUCCESS - saved https://www.bing.com/th?id=OIP.JZ0cA5Cm8aKCo4ygw2lh9wHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\battlefield_4_screenshot\8ca7031cfd.jpg
the_elder_scrolls_online__tamriel_unlimited_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.FmKvt01yRVcH-2PpVG0VgAHaEK&w=300&h=168&c=7&o=5&pid=1.7 - as ./images\the_elder_scrolls_online__tamriel_unlimited_screenshot\3e3206315d.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.v6xeJwxsxHttdOop2WINvQHaEn&w=272&h=169&c=7&o=5&pid=1.7 - as ./images\the_elder_scrolls_online__tamriel_unlimited_screenshot\2d8b70898d.jpg
mad_max_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.Tm3kjEwuSG1TdAI6gYhhTgHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\mad_max_screenshot\d138950c8c.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.rx3IgGmLFBd8rFajY9BqfQH

Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.G4skJBOpE8K5eT1bnc8zPwHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\observer_screenshot\8819209d38.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.9XLFFb9YxltCSIIhfv8TXQHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\observer_screenshot\06405de0e9.jpg
efootball_pes_2020_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.6Yawjwp8YdEexY9tcgLSvwHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\efootball_pes_2020_screenshot\599179b31f.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.FfJtkiUU9oe08p67bt3L0wHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\efootball_pes_2020_screenshot\dcd7c13a2f.jpg
outlast_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.bLOyKcSdyJ-jR

Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.eaTRWi3JQmJPSVJ5UunfpgHaEg&w=300&h=179&c=7&o=5&pid=1.7 - as ./images\this_war_of_mine_screenshot\17e74b9d5c.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP._xP9xrlx0XR-D7apJ8B46AHaEK&w=300&h=168&c=7&o=5&pid=1.7 - as ./images\this_war_of_mine_screenshot\33b4849106.jpg
total_war__three_kingdoms_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.bh1efcP-uy7mu7Z3kdfVtAHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\total_war__three_kingdoms_screenshot\ec38505daf.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.U5WE1c5wrhwRaLA99RT6WgHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\total_war__three_kingdoms_screenshot\8095a949c3.jpg
i.g.i._2__covert_strike_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS

...

Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.bcU4UYdIqiD5lEKtVJGXbwHaF7&w=230&h=184&c=7&o=5&pid=1.7 - as ./images\silent_hill_2_screenshot\dff0c6f3b1.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.htOpfoY2gymc9wKDwtl3_AHaFj&w=245&h=184&c=7&o=5&pid=1.7 - as ./images\silent_hill_2_screenshot\1c56331da7.jpg
uncharted_3__drake's_deception_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.mVyOjD3KYOrwRj3jHb9dCgHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\uncharted_3__drake's_deception_screenshot\843a42c5c0.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.Xt0WWowUODMLOV2jqzUM4wHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\uncharted_3__drake's_deception_screenshot\28c9e7cf3f.jpg
yakuza_4_remastered_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SU

SUCCESS - saved https://www.bing.com/th?id=OIP.XvijWmRoD8-iyUl3woUgNQHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\shadow_warrior_2_screenshot\e73b6254a2.jpg
tomb_raider__legend_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP._R9o_t6PqvN6nAEh2bBdowHaFj&w=288&h=206&c=7&o=5&pid=1.7 - as ./images\tomb_raider__legend_screenshot\ec9686f93e.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.iDcnFif8WxOmVFY_R5HH6gHaF7&w=275&h=206&c=7&o=5&pid=1.7 - as ./images\tomb_raider__legend_screenshot\5539607c59.jpg
neighbours_from_hell_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.aWsA7dMGnuC1F7Sk2kqWNwHaEK&w=267&h=160&c=7&o=5&pid=1.7 - as ./images\neighbours_from_hell_screenshot\80199bd0a9.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.zcC4rwIFwaZR7THILSYGSwHaFj&w=210&h=160&c=7&o=5&pid=1.7 - as ./imag

Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.AQiT6WlVOo9lkk5ePm3I1wHaFj&w=233&h=175&c=7&o=5&pid=1.7 - as ./images\dungeon_siege_ii_screenshot\b27eadf8e3.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.sht6HYNBJymouGtBo_wjuAHaFj&w=233&h=175&c=7&o=5&pid=1.7 - as ./images\dungeon_siege_ii_screenshot\fb32e86c99.jpg
spec_ops__the_line_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.geWUpn8CUmQ-pyv3ADd5MwHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\spec_ops__the_line_screenshot\f83ca78d41.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.2idcC47wxBnZmX73LMX2RAHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\spec_ops__the_line_screenshot\8f6e51a238.jpg
ryse__son_of_rome_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.c

SUCCESS - saved https://www.bing.com/th?id=OIP.glHtElGAmuiVapTpU7B1JwHaEK&w=231&h=160&c=7&o=5&pid=1.7 - as ./images\men_of_war_screenshot\11bb9a46ef.jpg
grand_ages__medieval_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.bM4Edl0OeVkMmzsb_EG3VgHaEK&w=293&h=164&c=7&o=5&pid=1.7 - as ./images\grand_ages__medieval_screenshot\cef968d5f9.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.PIQCCiYjjncjCtOvj4rFdAHaEF&w=298&h=164&c=7&o=5&pid=1.7 - as ./images\grand_ages__medieval_screenshot\9bce095ba2.jpg
bed_wars_screenshot
Found: 35 search results. Extracting links from 0:35
1
2
2
2
Found: 2 image links, done!
SUCCESS - saved https://www.bing.com/th?id=OIP.VD3mU-doaTqmd19JMPbG2wHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\bed_wars_screenshot\b89def8270.jpg
SUCCESS - saved https://www.bing.com/th?id=OIP.a3B8w_BLqFujCVQ0apGY6AHaEK&w=294&h=165&c=7&o=5&pid=1.7 - as ./images\bed_wars_screenshot\33c0

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".ksb"}
  (Session info: chrome=78.0.3904.108)
